In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import glob
import seaborn as sns
import spacy
import string
import re
import nltk
from spacy.symbols import ORTH
import xgboost as xgb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
whole_clean_df = pd.read_csv("/content/drive/MyDrive/NLP_Group 19/whole_clean_df.csv")

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(whole_clean_df[['lemma']],whole_clean_df['Emotion'],test_size=0.33,random_state=15)

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stops=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
whole_clean_df.head()

,Unnamed: 0,id,user_location,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,Country,text_no_sw,tokenized_text,Tweet_stemmed,lemma,Emotion,scores,neg,neu,pos,compound,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines
0,0,1340539111971516416,La Crescenta Montrose CA,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0,False,USA,same folks said daikon paste could treat cytok...,"['same', 'folks', 'said', 'daikon', 'paste', '...","['same', 'folk', 'said', 'daikon', 'past', 'co...","['same', 'folk', 'said', 'daikon', 'paste', 'c...",anger,"{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'comp...",0.000,0.769,0.231,0.4019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1338158543359250433,San Francisco CA,834,666,178,False,2020-12-13 16:27:13,While the world has been on the wrong side of ...,NaN,Twitter Web App,1,1,False,USA,while world wrong side history year hopefully ...,"['while', 'world', 'wrong', 'side', 'history',...","['while', 'world', 'wrong', 'side', 'histori',...","['while', 'world', 'wrong', 'side', 'history',...",positive,"{'neg': 0.196, 'neu': 0.633, 'pos': 0.171, 'co...",0.196,0.633,0.171,-0.1027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1337855739918835717,Vancouver BC Canada,49165,3933,21853,True,2020-12-12 20:23:59,Facts are immutable Senator even when youre no...,NaN,Twitter Web App,446,2129,False,Canada,facts immutable senator even youre ethically s...,"['facts', 'immutable', 'senator', 'even', 'you...","['fact', 'immut', 'senat', 'even', 'your', 'et...","['fact', 'immutable', 'senator', 'even', 'your...",positive,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1337851215875608579,Austria Ukraine and Kosovo,2731,5001,69344,False,2020-12-12 20:06:00,it is a bit sad to claim the fame for success ...,['vaccination'],Twitter Web App,0,4,False,Austria,bit sad claim fame success vaccination patriot...,"['bit', 'sad', 'claim', 'fame', 'success', 'va...","['bit', 'sad', 'claim', 'fame', 'success', 'va...","['bit', 'sad', 'claim', 'fame', 'success', 'va...",positive,"{'neg': 0.175, 'neu': 0.452, 'pos': 0.373, 'co...",0.175,0.452,0.373,0.5423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1337841934170255365,Turkey Israel,1302,78,339,False,2020-12-12 19:29:07,while deaths are closing in on the mark milli...,"['PfizerBioNTech', 'Vaccine']",Twitter Web App,0,0,False,Israel,deaths closing mark millions people wait pfize...,"['deaths', 'closing', 'mark', 'millions', 'peo...","['death', 'close', 'mark', 'million', 'peopl',...","['death', 'closing', 'mark', 'million', 'peopl...",positive,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# tf_idf_vectorizer = TfidfVectorizer(stop_words='english')
# tfidf_simple = tf_idf_vectorizer.fit(X_train)

In [ ]:
# def preprocessor(data):
#   from sklearn.feature_extraction.text import TfidfVectorizer
#   new_tfidf_df = tfidf_simple.transform(data)
#   new_tfidf_df = new_tfidf_df.todense()
#   return new_tfidf_df

In [ ]:
X_traintf, X_testtf, Y_traintf, Y_testtf = train_test_split(whole_clean_df[['text_no_sw']],whole_clean_df['Emotion'],test_size=0.33,random_state=15)

In [ ]:
vectorizer= TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_traintf['text_no_sw'])
tf_x_test = vectorizer.transform(X_testtf['text_no_sw'])

In [ ]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE(k_neighbors=1,random_state=35)
tf_x_train_smote, Y_traintf_smote = oversample.fit_resample(X=tf_x_train, y=Y_traintf)

In [ ]:
# Y_testtf = LabelEncoder().fit_transform(Y_testtf)
# Y_traintf = LabelEncoder().fit_transform(Y_traintf)

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters_1_tf = {'num_leaves':[20,40,60,80,100]}
lgb_tf1 = lgb.LGBMClassifier(learning_rate=.5,min_child_samples=5,reg_alpha=.01,max_depth=5)
clf_tf=GridSearchCV(lgb_tf1,parameters_1_tf,scoring='accuracy',cv=StratifiedKFold(n_splits=3))
clf_tf.fit(tf_x_train, Y_traintf)
print(clf_tf.best_params_)

{'num_leaves': 20}


In [ ]:
from sklearn.model_selection import GridSearchCV
parameters_tf2 = {'min_child_samples':[5,10,15]}
lgb_tf2 = lgb.LGBMClassifier(learning_rate=.5,reg_alpha=.01,max_depth=5,num_leaves=20)
clf_tf2=GridSearchCV(lgb_tf2,parameters_tf2,scoring='accuracy',cv=StratifiedKFold(n_splits=3))
clf_tf2.fit(tf_x_train, Y_traintf)
print(clf_tf2.best_params_)


{'min_child_samples': 15}


In [ ]:
from sklearn.model_selection import GridSearchCV
parameters_tf3 =  {'max_depth':[-1,5,10,20]}
lgb_tf3 = lgb.LGBMClassifier(learning_rate=.5,reg_alpha=.01,num_leaves=20,min_child_samples=15)
clf_tf3=GridSearchCV(lgb_tf3,parameters_tf3,scoring='accuracy',cv=StratifiedKFold(n_splits=3))
clf_tf3.fit(tf_x_train, Y_traintf)
print(clf_tf3.best_params_)
 



{'max_depth': 5}


In [ ]:
from sklearn.model_selection import GridSearchCV
parameters_tf4 = {'reg_alpha':[0,0.01,0.03]}
lgb_tf4 = lgb.LGBMClassifier(learning_rate=.5,max_depth=5,num_leaves=20,min_child_samples=15)
clf_tf4=GridSearchCV(lgb_tf4,parameters_tf4,scoring='accuracy',cv=StratifiedKFold(n_splits=3))
clf_tf4.fit(tf_x_train, Y_traintf)
print(clf_tf4.best_params_)


{'reg_alpha': 0.03}


In [ ]:
from sklearn.model_selection import GridSearchCV
parameters_tf5 = {'learning_rate':[0.05,0.1,0.2]}
lgb_tf5 = lgb.LGBMClassifier(min_child_samples=15,reg_alpha=.03,max_depth=5,num_leaves=20)
clf_tf5=GridSearchCV(lgb_tf5,parameters_tf5,scoring='accuracy',cv=StratifiedKFold(n_splits=3))
clf_tf5.fit(tf_x_train, Y_traintf)
print(clf_tf5.best_params_)
 



{'learning_rate': 0.1}


In [ ]:
lgb_tf_def = lgb.LGBMClassifier(objective='multiclass')#,class_weight=class_weight)
lgb_tf_def.fit(tf_x_train,Y_traintf)

LGBMClassifier(objective='multiclass')

In [ ]:
y_pred_tf_def = lgb_tf_def.predict(tf_x_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_testtf, y_pred_tf_def, digits=5))

              precision    recall  f1-score   support

       anger    0.76497   0.74919   0.75699       921
anticipation    0.72951   0.75637   0.74270       706
     disgust    0.25000   0.08333   0.12500        12
        fear    0.84722   0.79559   0.82059      4168
         joy    0.00000   0.00000   0.00000         1
    negative    0.82422   0.54417   0.65553      1551
    positive    0.83345   0.92417   0.87647     10326
     sadness    0.18367   0.19149   0.18750        47
    surprise    0.77299   0.60449   0.67844       445
       trust    0.85968   0.80352   0.83065      4209

    accuracy                        0.83034     22386
   macro avg    0.60657   0.54523   0.56739     22386
weighted avg    0.83129   0.83034   0.82718     22386



In [ ]:
roc_auc_score(Y_testtf, lgb_tf_def.predict_proba(tf_x_test), multi_class='ovo')

0.8855822268646389

In [ ]:
lgb_tf_def2 = lgb.LGBMClassifier(objective='multiclass')#,class_weight = 'balanced')#,class_weight=class_weight)
lgb_tf_def2.fit(tf_x_train_smote,Y_traintf_smote)

LGBMClassifier(objective='multiclass')

In [ ]:
y_pred_tf_def2 = lgb_tf_def2.predict(tf_x_test)


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_testtf, y_pred_tf_def2, digits=5))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



              precision    recall  f1-score   support

       anger    0.72693   0.84691   0.78235       921
anticipation    0.59647   0.86261   0.70527       706
     disgust    0.00000   0.00000   0.00000        12
        fear    0.84132   0.81286   0.82685      4168
         joy    0.00000   0.00000   0.00000         1
    negative    0.75182   0.66602   0.70632      1551
    positive    0.92865   0.85202   0.88869     10326
     sadness    0.76000   0.40426   0.52778        47
    surprise    0.66850   0.82022   0.73663       445
       trust    0.74943   0.86268   0.80208      4209

    accuracy                        0.83190     22386
   macro avg    0.60231   0.61276   0.59760     22386
weighted avg    0.84160   0.83190   0.83380     22386



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_testtf, lgb_tf_def2.predict_proba(tf_x_test), multi_class='ovo')

0.9581491062305428

In [ ]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
param_test ={'num_leaves': range(20, 170, 15), 
             
             'max_depth': range(3,10,2),
             'min_child_samples': sp_randint(50, 300), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [7, 10, 50, 100],
             'reg_lambda': [10, 20, 50, 100,120],
             'n_estimators':[100,200,300,400]}

In [ ]:
n_HP_points_to_test = 8

import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

#n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 5000 define only the absolute maximum
clf = lgb.LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=2,colsample_bytree=0.9731668400523877)#, n_estimators=600)#,class_weight=class_weight)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    scoring='accuracy',
    cv=StratifiedKFold(n_splits=5),
    #refit=True,
    random_state=314,
    verbose=1)

In [ ]:
gs.fit(tf_x_train, Y_traintf)#, **fit_params)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning:

The least populated class in y has only 3 members, which is less than n_splits=5.



RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
                   estimator=LGBMClassifier(colsample_bytree=0.9731668400523877,
                                            metric='None', n_jobs=2,
                                            random_state=314),
                   n_iter=8,
                   param_distributions={'colsample_bytree': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f6e71006dd0>,
                                        'max_depth': range(3, 10, 2),
                                        'min_child_samples': <sc...v_frozen object at 0x7f6e70a4a790>,
                                        'min_child_weight': [1e-05, 0.001, 0.01,
                                                             0.1, 1, 10.0,
                                                             100.0, 1000.0,
                                                             10000.0],
                                        'n_estimators': [100, 200

In [ ]:
gs.best_params_

{'colsample_bytree': 0.55607546409401,
 'max_depth': 9,
 'min_child_samples': 215,
 'min_child_weight': 0.1,
 'n_estimators': 400,
 'num_leaves': 95,
 'reg_alpha': 7,
 'reg_lambda': 10,
 'subsample': 0.6881882366816626}

In [ ]:
lgb_tf_def2_tune = lgb.LGBMClassifier(objective='multiclass',colsample_bytree=0.55607546409401,min_child_samples=215,min_child_weight=0.1,num_leaves=95,reg_alpha=7,reg_lambda=10,subsample=0.6881882366816626,n_estimators=400,max_depth=9)#,class_weight = 'balanced')#,class_weight=class_weight)
lgb_tf_def2_tune.fit(tf_x_train_smote,Y_traintf_smote)

LGBMClassifier(colsample_bytree=0.55607546409401, max_depth=9,
               min_child_samples=215, min_child_weight=0.1, n_estimators=400,
               num_leaves=95, objective='multiclass', reg_alpha=7,
               reg_lambda=10, subsample=0.6881882366816626)

In [ ]:
pred_tune  = lgb_tf_def2_tune.predict(tf_x_test)

In [ ]:
print(classification_report(Y_testtf, pred_tune, digits=5))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



              precision    recall  f1-score   support

       anger    0.72306   0.77959   0.75026       921
anticipation    0.61910   0.83569   0.71127       706
     disgust    0.00000   0.00000   0.00000        12
        fear    0.81084   0.74664   0.77742      4168
         joy    0.00000   0.00000   0.00000         1
    negative    0.62892   0.56931   0.59763      1551
    positive    0.85236   0.84147   0.84688     10326
     sadness    0.77778   0.44681   0.56757        47
    surprise    0.69710   0.75506   0.72492       445
       trust    0.71502   0.76360   0.73851      4209

    accuracy                        0.78455     22386
   macro avg    0.58242   0.57382   0.57145     22386
weighted avg    0.78691   0.78455   0.78455     22386



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
lgb_tf = lgb.LGBMClassifier(reg_alpha=.03,num_leaves=20,min_child_samples=15,max_depth=5,learning_rate=.1,objective='multiclass')#,class_weight=class_weight)
lgb_tf.fit(tf_x_train,Y_traintf)

LGBMClassifier(max_depth=5, min_child_samples=15, num_leaves=20,
               objective='multiclass', reg_alpha=0.03)

In [ ]:
y_pred_tf = lgb_tf.predict(tf_x_test)

In [ ]:
y_pred_tf

array(['positive', 'positive', 'fear', ..., 'trust', 'positive',
       'positive'], dtype=object)

In [ ]:
# print(y_pred_tf.tolist())

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_testtf, y_pred_tf, digits=5))

              precision    recall  f1-score   support

       anger    0.73893   0.70684   0.72253       921
anticipation    0.66281   0.64873   0.65569       706
     disgust    0.14286   0.08333   0.10526        12
        fear    0.81083   0.67155   0.73465      4168
         joy    0.00000   0.00000   0.00000         1
    negative    0.81660   0.39329   0.53090      1551
    positive    0.73558   0.92136   0.81806     10326
     sadness    0.12766   0.25532   0.17021        47
    surprise    0.75516   0.57528   0.65306       445
       trust    0.83086   0.63839   0.72202      4209

    accuracy                        0.75887     22386
   macro avg    0.56213   0.48941   0.51124     22386
weighted avg    0.76972   0.75887   0.75046     22386



In [ ]:
def get_non_stopwords(review):
    """Returns a list of non-stopwords"""
    return {x:1 for x in spacy_tok(str(review).lower()) if x not in stops}.keys()

In [ ]:
def load_word_embeddings(file="/content/drive/MyDrive/NLP_Group 19/glove.6B.200d.txt"):
    embeddings={}
    with open(file,'r',encoding='UTF-8') as infile:
        for line in infile:
            values=line.split()
            embeddings[values[0]]=np.asarray(values[1:],dtype='float32')
    return embeddings

In [ ]:
embeddings = load_word_embeddings()

In [ ]:
def sentence_features_v2(s, embeddings=embeddings,emb_size=200):
    # ignore stop words
    #words=get_non_stopwords(s)
    words=[w for w in s if w.isalpha() and w in embeddings]
    if len(words)==0:
        return np.hstack([np.zeros(emb_size)])
    M=np.array([embeddings[w] for w in words])
    return M.mean(axis=0)

In [ ]:
w = sentence_features_v2(X_train['lemma'])
w.shape

(200,)

In [ ]:
x_train = np.array([sentence_features_v2(x) for x in X_train['lemma']])
x_test = np.array([sentence_features_v2(x) for x in X_test['lemma']])

In [ ]:
x_train_csv = pd.DataFrame(x_train)

In [ ]:
x_test_csv = pd.DataFrame(x_test)

In [ ]:
x_train_csv.to_csv('x_train.csv')
#x_test_csv.to_csv('x_test.csv')

In [ ]:
x_train.shape

(45448, 200)

In [ ]:
Y_train

28037        positive
3974            trust
54657        positive
60344            fear
50323        negative
             ...     
35483    anticipation
49015        negative
8076         negative
52981        positive
7624         surprise
Name: Emotion, Length: 45448, dtype: object

In [ ]:
Y_train

array([6, 9, 6, ..., 5, 6, 8])

In [ ]:
#Y_train = LabelEncoder().fit_transform(Y_train)

In [ ]:

#Y_test = LabelEncoder().fit_transform(Y_test)

In [ ]:
len(Y_train)

45448

In [ ]:
Y_test

array([3, 6, 3, ..., 9, 9, 6])

In [ ]:
import lightgbm as lgb
# 

In [ ]:
lgb_clf = lgb.LGBMClassifier()
lgb_clf.fit(x_train, Y_train)
#y_pred = lgb_clf.predict(x_test.to_list())
#accuracy_dict[category] = accuracy_score(y_test[category], y_pred)*100

LGBMClassifier()

In [ ]:
y_pred = lgb_clf.predict(x_test)

In [ ]:
print(classification_report(Y_test, y_pred, digits=5))

              precision    recall  f1-score   support

           0    0.50951   0.14549   0.22635       921
           1    0.52941   0.05099   0.09302       706
           2    0.00000   0.00000   0.00000        12
           3    0.56017   0.34285   0.42536      4168
           4    0.00000   0.00000   0.00000         1
           5    0.72174   0.16054   0.26266      1551
           6    0.56332   0.91584   0.69757     10326
           7    0.03175   0.08511   0.04624        47
           8    0.88793   0.23146   0.36720       445
           9    0.67998   0.31551   0.43103      4209

    accuracy                        0.56911     22386
   macro avg    0.44838   0.22478   0.25494     22386
weighted avg    0.59737   0.56911   0.51985     22386



In [ ]:
len(y_pred)

22386

In [ ]:
len(Y_test)

22386

In [ ]:
Y_train

array([6, 9, 6, ..., 5, 6, 8])

In [ ]:
Y_test

array([3, 6, 3, ..., 9, 9, 6])

In [ ]:
y_pred

array([6, 6, 3, ..., 9, 6, 6])

In [ ]:
X_train

,lemma
28037,"['shocked', 'learn', 'dr', 'kk', 'aggarwal', '..."
3974,"['i', 'havent', 'keepingup', 'latest', 'news',..."
54657,"['zycovd', 'dnavaccine', 'india', 'need', 'dna..."
60344,"['wheres', 'moderna', 'boost', 'shot']"
50323,"['mayo', 'clinic', 'covid', 'breakthrough', 'r..."
...,...
35483,"['sante', 'say', 'newest', 'shipment', 'modern..."
49015,"['it', 'seems', 'live', 'pluto', 'incorrect', ..."
8076,"['simonyan', 'canada', 'politicizing', 'vaccin..."
52981,"['sputnikv', 'bilderberggroup', 'trilateralcom..."


In [ ]:
import time
start = time.time()
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgb


In [ ]:
len(x_train)

45448

In [ ]:
#Y_muti_train = LabelEncoder().fit_transform(Y_train)

In [ ]:
#Y_muti_train


In [ ]:
Y_train

array([6, 9, 6, ..., 5, 6, 8])

In [ ]:
lgb_glove = lgb.LGBMClassifier()

In [ ]:
lgb = lgb.LGBMClassifier(learning_rate=.5,min_child_samples=5,reg_alpha=.01,max_depth=5)

#Leaves :40

In [ ]:
parameters_1 = {'num_leaves':[20,40,60,80,100]}#, 'min_child_samples':[5,10,15],'max_depth':[-1,5,10,20],
            # 'learning_rate':[0.05,0.1,0.2],'reg_alpha':[0,0.01,0.03]}

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
clf=GridSearchCV(lgb,parameters_1,scoring='accuracy',cv=StratifiedKFold(n_splits=3))
clf.fit(X=x_train, y=Y_train)
print(clf.best_params_)
#predicted_=clf.predict(X_test)
#print('Classification of the result is:')
#print(accuracy_score(Y_test, predicted))


{'num_leaves': 20}


# Min_child_samples:

In [ ]:
import lightgbm as lgb

#lgb = lgb.LGBMClassifier(learning_rate=.5,min_child_samples=5,reg_alpha=.01,max_depth=5)
lgb2 = lgb.LGBMClassifier(learning_rate=.5,reg_alpha=.01,max_depth=5,num_leaves=40)

In [ ]:
parameters_2 = {'min_child_samples':[5,10,15]}#,'max_depth':[-1,5,10,20],
            # 'learning_rate':[0.05,0.1,0.2],'reg_alpha':[0,0.01,0.03]}

In [ ]:
from sklearn.model_selection import StratifiedKFold
clf2=GridSearchCV(lgb2,parameters_2,scoring='accuracy',cv=StratifiedKFold(n_splits=3))
clf2.fit(X=x_train, y=Y_train)
print(clf2.best_params_)

#max_depth:10

In [ ]:
import lightgbm as lgb

#lgb = lgb.LGBMClassifier(learning_rate=.5,min_child_samples=5,reg_alpha=.01,max_depth=5)
lgb3 = lgb.LGBMClassifier(learning_rate=.5,reg_alpha=.01,num_leaves=40,min_child_samples=10)

In [ ]:
parameters_3 = {'max_depth':[-1,5,10,20]}
            # 'learning_rate':[0.05,0.1,0.2],'reg_alpha':[0,0.01,0.03]}

In [ ]:
from sklearn.model_selection import StratifiedKFold
clf3=GridSearchCV(lgb3,parameters_3,scoring='accuracy',cv=StratifiedKFold(n_splits=3))
clf3.fit(X=x_train, y=Y_train)
print(clf3.best_params_)

#Reg_alpha: .03

In [ ]:
parameters_4 = {'reg_alpha':[0,0.01,0.03]}

In [ ]:
lgb4 = lgb.LGBMClassifier(learning_rate=.5,num_leaves=40,min_child_samples=10,max_depth=10)

In [ ]:
from sklearn.model_selection import StratifiedKFold
clf4=GridSearchCV(lgb4,parameters_4,scoring='accuracy',cv=StratifiedKFold(n_splits=3))
clf4.fit(X=x_train, y=Y_train)
print(clf4.best_params_)

In [ ]:
parameters_5 = {'learning_rate':[0.05,0.1,0.2]}

In [ ]:
lgb5 = lgb.LGBMClassifier(reg_alpha=.03,num_leaves=40,min_child_samples=10,max_depth=10)

In [ ]:
from sklearn.model_selection import StratifiedKFold
clf5=GridSearchCV(lgb4,parameters_5,scoring='accuracy',cv=StratifiedKFold(n_splits=3))
clf5.fit(X=x_train, y=Y_train)
print(clf5.best_params_)

In [ ]:
lgb6 = lgb.LGBMClassifier(reg_alpha=.03,num_leaves=40,min_child_samples=10,max_depth=10,learning_rate=.05)
lgb6.fit(X=x_train, y=Y_train)

In [ ]:
y_pred2 = lgb6.predict(x_test)

In [ ]:
y_pred2

In [ ]:
Y_test

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred2, digits=5))

In [ ]:
class_weight = {0:2.4646420824,
                1:3.1983110486,
                2:239.2,
                3:0.5398907104,
                4:1514.9333333333,
                5:1.4215827338,
                6:0.2145392749,
                7:77.0305084746,
                8:5.0330011074,

                9:0.541047619

}

In [ ]:
lgb7 = lgb.LGBMClassifier(reg_alpha=.03,num_leaves=40,min_child_samples=10,max_depth=10,learning_rate=.05,class_weight=class_weight)
lgb7.fit(X=x_train, y=Y_train)

In [ ]:
y_pred3 = lgb7.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred3, digits=5))